# Bring Your Own Checkpoint (BYOC) to SamabaStudio 

Example checkpoint download from Huggingface

In [ ]:
import os
from huggingface_hub import hf_hub_download, HfApi

model_name = "NousResearch/Hermes-2-Theta-Llama-3-8B"  # Replace with your desired model

target_dir = "./models"  
os.makedirs(target_dir, exist_ok=True)

repo_files = HfApi().list_repo_files(model_name)

for file_name in repo_files:
    hf_hub_download(repo_id=model_name, filename=file_name, cache_dir=target_dir)

After download, modify [config.yaml](./config.yaml) file with the snapshot of the model and required parameters

check at the `chat_template` key in the [tokenizer_config.json](./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725/tokenizer_config.json) file inside the model snapshot and modify it to follow a chat jinja template like this:

```json
  "chat_template": "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|im_start|>'' + message['role'] + '\n'+ message['content'] | trim + '<|im_end|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"
```

Set BYOC object with checkpoints to upload config

In [1]:
from src.snsdk_byoc_wrapper import BYOC
byoc = BYOC("./config.yaml")

2024-11-05 09:32:02,869 [INFO] Using config file located in ./config.yaml
2024-11-05 09:32:02,876 [INFO] Using variables from Snapi config to set up Snsdk.


Get params from checkpoints paths set in [config.yaml](./config.yaml) file

In [8]:
byoc.find_config_params(update_config_file=True)

2024-11-05 09:22:23,299 [INFO] Params for checkpoint in ./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725:
[{'model_arch': 'llama', 'seq_length': 8192, 'vocab_size': 128256}]
2024-11-05 09:22:23,300 [INFO] config updated with checkpoints parameters
2024-11-05 09:22:23,306 [INFO] config file updated with checkpoints parameters


Find suitable SambaStudio apps for your checkpoints

In [9]:
byoc.get_suitable_apps()

2024-11-05 09:28:18,067 [INFO] Checkpoint HermesProInstructV10 suitable apps:
{'id': '61fa0993-04a2-42ca-9db1-1eff693ea978', 'name': 'Samba1 Llama3 Experts'}
{'id': 'ad39e323-9878-4914-8e29-82c9f2939475', 'name': 'Llama 3'}


Set in the `app_id` key of your checkpoint in the [config.yaml](./config.yaml) file the desired app_id of your checkpoints

Upload the checkpoints in your [config.yaml](./config.yaml) file to SambaStudio (this could take from minutes to hours depending on the size of your checkpoints)

In [10]:
#load the updated config.yaml file including app ids and model params
byoc = BYOC("./config.yaml")

2024-11-04 13:44:12,763 [INFO] Using config file located in ./config.yaml
2024-11-04 13:44:12,765 [INFO] Using variables from Snapi config to set up Snsdk.


In [4]:
byoc.upload_checkpoints()

2024-11-01 15:01:30,911 [INFO] Model with name 'HermesProInstructv10' not found
2024-11-01 15:01:31,098 [INFO] App with name '61fa0993-04a2-42ca-9db1-1eff693ea978' found with id 61fa0993-04a2-42ca-9db1-1eff693ea978
2024-11-01 16:53:44,293 [ERROR] Error uploading model checkpoint Process returned a non-zero exit code.
2024-11-01 16:53:44,296 [ERROR] urllib3.exceptions.SSLError: EOF occurred in violation of protocol (_ssl.c:2423)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/jorgep/Documents/ask_public_own/snapienv/lib/python3.11/site-packages/requests/adapters.py", line 667, in send
    resp = conn.urlopen(
           ^^^^^^^^^^^^^
  File "/Users/jorgep/Documents/ask_public_own/snapienv/lib/python3.11/site-packages/urllib3/connectionpool.py", line 843, in urlopen
    retries = retries.increment(
              ^^^^^^^^^^^^^^^^^^
  File "/Users/jorgep/Documents/ask_public_own/snapienv/lib/python3.11/site-packages/u

In [2]:
byoc.upload_checkpoints()

2024-11-01 11:01:06,370 [INFO] Model with name 'HermesProInstructV10' found with id 91c93f34-5ba4-4248-be6f-7aba79da3689
2024-11-01 11:01:06,373 [INFO] Model checkpoint with name 'HermesProInstructV10' not created it already exist with id 91c93f34-5ba4-4248-be6f-7aba79da3689
2024-11-01 11:01:06,375 [INFO] Checkpoint for model HermesProInstructV10 finished successfully with result 91c93f34-5ba4-4248-be6f-7aba79da3689 


You can check the status of the checkpoints after the upload process

In [2]:
byoc.get_checkpoints_status()

2024-11-04 10:58:47,648 [INFO] model HermesProInstructV10 status: 
 {'model_id': '91c93f34-5ba4-4248-be6f-7aba79da3689', 'status': 'Available', 'progress': 100, 'stage': 'convert', 'status_code': 200}


We can create a bundle model including uploaded and existing checkpoints in sambastudio setting `composite_model` params in [config.yaml](./config.yaml) file

In [2]:
byoc.create_composite_model()

2024-11-01 08:31:23,695 [INFO] Project with name 'TestByocCoE' not found
2024-11-01 08:31:23,879 [INFO] Model with name 'Meta-Llama-3-8B-Instruct' found with id b16efa3a-90a9-4708-b963-ffd5306780e5
2024-11-01 08:31:24,077 [INFO] Model with name 'e5-mistral-7b-instruct' found with id 386f59fc-466f-4f2c-a681-72bfefe8b21d
2024-11-01 08:31:24,256 [INFO] Model with name 'HermesProInstructV10' found with id 91c93f34-5ba4-4248-be6f-7aba79da3689
2024-11-01 08:31:24,256 [INFO] Models to include in composite found with ids '[('Meta-Llama-3-8B-Instruct', 'b16efa3a-90a9-4708-b963-ffd5306780e5'), ('e5-mistral-7b-instruct', '386f59fc-466f-4f2c-a681-72bfefe8b21d'), ('HermesProInstructV10', '91c93f34-5ba4-4248-be6f-7aba79da3689')]
2024-11-01 08:31:24,654 [INFO] Composite model with name TestByocCoE created with id 8cdfe3f0-651f-4611-a000-4ab6e7197eb3


'8cdfe3f0-651f-4611-a000-4ab6e7197eb3'

Then we can deploy the bundle model, setting `deployement` params in [config.yaml](./config.yaml) file

In [2]:
# create project
byoc.create_project()

2024-11-01 08:34:59,776 [INFO] Project with name 'example-byoc-project' not found
2024-11-01 08:34:59,949 [INFO] Project with name example-byoc-project created with id 9d17a0b5-b0f1-478c-9edb-51e8c93fd492


'9d17a0b5-b0f1-478c-9edb-51e8c93fd492'

In [2]:
# deploy your model in an endpoint
byoc.create_endpoint()

2024-11-05 09:32:13,279 [INFO] Project with name 'example-byoc-project' found with id 9d17a0b5-b0f1-478c-9edb-51e8c93fd492
2024-11-05 09:32:13,456 [INFO] Model with name 'TestByocCoE' found with id 8cdfe3f0-651f-4611-a000-4ab6e7197eb3
2024-11-05 09:32:13,621 [INFO] Endpoint with name 'test-endpoint-byoc' not created it already exist with id 0da58115-fbc7-4c81-9ec0-543980c49e06


'0da58115-fbc7-4c81-9ec0-543980c49e06'

In [12]:
# get endpoint details
byoc.get_endpoint_details()

2024-11-05 09:28:57,562 [INFO] Project with name 'example-byoc-project' found with id 9d17a0b5-b0f1-478c-9edb-51e8c93fd492


{'status': 'SettingUp',
 'url': '/api/predict/generic/9d17a0b5-b0f1-478c-9edb-51e8c93fd492/d10b6827-4d9a-4208-8018-648262fd4601',
 'langchain wrapper env': {'SAMBASTUDIO_URL': '**************************************',
  'SAMBASTUDIO_API_KEY': '***********************'}}

Use the models

In [11]:
from langchain_community.chat_models import ChatSambaStudio

endpoint_details = byoc.get_endpoint_details()
endpoint_url = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_URL")
endpoint_api_key = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_API_KEY")

llm = ChatSambaStudio(
    sambastudio_url=endpoint_url,
    sambastudio_api_key=endpoint_api_key,
    do_sample= True, 
    temperature= 0.01,
    max_tokens= 512,
    top_p= 1.0,
    top_k= 50,
    process_prompt=False,
    model= "HermesProInstructV10"
)
print(llm.invoke("tell me a joke").content)

2024-11-05 09:31:36,059 [INFO] Project with name 'example-byoc-project' found with id 9d17a0b5-b0f1-478c-9edb-51e8c93fd492


Here's one:

Why don't scientists trust atoms?

Because they make up everything!
